In [1]:
import pandas as pd

# import data
capacity_factors = pd.read_csv('../charts/state_capacity_factors.csv')
us_electric_gen = pd.read_csv('../electricity_generation_extractor/us_electric_generation_2001_20.csv')

In [2]:
# using this object to align our us state names with their abbreviation
# code copied from https://gist.github.com/rogerallen/1583593
# to invert it, simply run `dict(map(reversed, us_state_to_abbrev.items()))`
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
} 

# let's just fix align the state names between the data
us_electric_gen['state'] = us_electric_gen['state'].replace(dict(map(reversed, us_state_to_abbrev.items())))
us_electric_gen['state'] = us_electric_gen['state'].apply(lambda state: state.lower().replace(' ', '_'))

# fix the states in capacity factors
capacity_factors['state'] = capacity_factors['state'].apply(lambda state: state.lower().replace(' ', '_'))

In [3]:
# let's just subset to the columns we need
green_electric_gen_cols = ['state', 'year', 'all_fuels', 'hydro_electric', 
                           'hydro_electric_storage', 'all_solar', 'wind', 'other_renewables']
green_electric_df = us_electric_gen.loc[:, green_electric_gen_cols]

# also we don't have capacity factors (yet) for HI & AK, so I'll subset them out
green_electric_df = green_electric_df.loc[~green_electric_df['state'].isin(['alaska', 'hawaii'])]

In [4]:
### for example's sake, let's just keep 2020 data, since the static capacity factors have been built more recently
green_electric_df_2020_only = green_electric_df.loc[green_electric_df['year']==2020]

In [5]:
renewables = ['all_solar', 'wind', 'hydro_electric', 'hydro_electric_storage', 'other_renewables']

green_electric_df_2020_only['total_renewable'] = green_electric_df_2020_only.loc[:, renewables].sum(axis=1)

# calculate the total generation between all_fuels & total_renewables
green_electric_df_2020_only['total_generation'] = green_electric_df_2020_only['all_fuels'] + green_electric_df_2020_only['total_renewable']

# calculate % of generation from renewables
green_electric_df_2020_only['%_gen_renewable'] = round(green_electric_df_2020_only['total_renewable'] / green_electric_df_2020_only['total_generation'],2)

<ipython-input-5-0d089c22c18d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_electric_df_2020_only['total_renewable'] = green_electric_df_2020_only.loc[:, renewables].sum(axis=1)
<ipython-input-5-0d089c22c18d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_electric_df_2020_only['total_generation'] = green_electric_df_2020_only['all_fuels'] + green_electric_df_2020_only['total_renewable']
<ipython-input-5-0d089c22c18d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [39]:
# I don't think the %_gen_renewable is totally right... but for now, let's work with this

# alright,

,state,year,all_fuels,hydro_electric,hydro_electric_storage,all_solar,wind,other_renewables,total_renewable,total_generation,%_gen_renewable
20,alabama,2020,137542.70160,13349.13400,0.000,383.57643,0.00000,3675.18229,17407.89272,154950.59432,0.11
40,arkansas,2020,54641.25866,4530.62500,65.579,358.38942,0.00000,1205.07693,6159.67035,60800.92901,0.10
60,arizona,2020,109305.05690,6423.60000,62.251,8749.49160,644.16000,6704.43653,22583.93913,131888.99603,0.17
80,california,2020,193074.93040,21377.47804,-37.126,47679.87321,13583.08900,60862.35423,143465.66848,336540.59888,0.43
100,colorado,2020,54115.01067,1668.56100,-151.391,2204.37961,13386.22800,15056.40315,32164.18076,86279.19143,0.37
120,connecticut,2020,41190.57249,325.94413,1.321,948.83825,12.18400,1007.62782,2295.91520,43486.48769,0.05
159,district_of_columbia,2020,123.23900,0.00000,0.000,0.00000,0.00000,0.00000,0.00000,123.23900,0.00
160,delaware,2020,5205.37200,0.00000,0.000,182.60627,5.17900,131.39400,319.17927,5524.55127,0.06
180,florida,2020,250827.79920,231.68000,0.000,7626.98194,1.42200,10534.98525,18395.06919,269222.86839,0.07
200,georgia,2020,120126.00060,4662.57477,-376.368,4109.26607,0.00000,9635.35619,18030.82903,138156.82963,0.13
